In [26]:
import warnings
warnings.filterwarnings("ignore")
import math, os, sys, copy, pickle
from skimage import io, filters, restoration
from skimage import morphology as morph
import PIL
from PIL import Image, ImageSequence, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import glob
import cv2
import multipagetiff as mtif
import tifffile

In [27]:
def plot(tif_files, try_denoise = False):
    fig, axes = plt.subplots(2, 4, figsize=(16,8), sharex=True, sharey=True)
    for num, tif_file in enumerate(tif_files):
        #print(tif_file, tif_file.rsplit(".", 1)[0][-1])
        # num = int(tif_file.rsplit(".", 1)[0][-1])-1
        im = Image.open(tif_file)
        if denoise:
            imarray = denoise(np.array(im))
        else:
            imarray = np.array(im)
        # print(num, num//4, num%4)
        axes[num//4, num%4].imshow(imarray)
        axes[num//4, num%4].set_title("{}".format(num+1))
    plt.show()
    # plt.colorbar()
#     fig.subplots_adjust(right=0.85)
#     cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
#     fig.colorbar(im, cax=cbar_ax)
def plot_oneZ(tif_files, z_pos = 0, z_num = 0):
    fig, axes = plt.subplots(2, 4, figsize=(16,8), sharex=True, sharey=True)
    for tif_file in tif_files:
        #print(tif_file, tif_file.rsplit(".", 1)[0][-1])
        num = int(tif_file.rsplit(".", 1)[0][-1])-1
        imgs = mtif.read_stack(tif_file)
        if z_num != 0:
            img = imgs[z_num - 1]
        else:
            img = imgs[math.floor(len(imgs) * z_pos)]
        # print(num, num//4, num%4)
        axes[num//4, num%4].imshow(img)
        axes[num//4, num%4].set_title("{}".format(num+1))
    plt.show()
def plot_single(img):
    #print(tif_file, tif_file.rsplit(".", 1)[0][-1])
    # im = Image.open(tif_file)
    # img = np.array(im)
    plt.figure()
    ax = plt.gca()
    im_vis = ax.imshow(img)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im_vis, cax=cax)
    plt.show()
def flatten_plot(tif_files):
    # fig, axes = plt.subplots(2,4, figsize=(16,8), sharex = True, sharey = True)
    plt.figure(figsize=(16,8))
    for i, tif_file in enumerate(tif_files):
        imgs = mtif.read_stack(tif_file)
        plt.subplot(2,4,i+1)
        img = mtif.flatten(imgs)
        plt.imshow(img)
    plt.tight_layout()

In [28]:
def normalize(img, min_range = 32, im_max = None, im_min = None, scale_to = 65535):
    img_new = copy.deepcopy(img)
    if im_max == None:
        im_max = np.max(img)
    if im_min == None:
        im_min = np.min(img)
    img_new = (img - im_min) / max([(im_max - im_min), min_range])
    img_new = np.array(img_new * scale_to).astype(np.uint16)
    return(img_new)
def filter(img, min_thr = 0, threshold = None):
    if threshold == None:
        try:
            threshold = max(filters.threshold_otsu(img), min_thr)
        except:
            threshold = img.min() + min_thr
    mask = img > threshold
    otsu_filtered = np.zeros_like(img)
    otsu_filtered[mask] = img[mask]
    return(otsu_filtered)
def filter_z(imgz, min_thr = 0, threshold = None):
    for i in range(len(imgz)):
        imgz[i] = filter(imgz[i], min_thr, threshold)
    return(imgz)
def sub_bg_thr(img, min_thr = 0, threshold = None):
    if threshold == None:
        try:
            threshold = max(filters.threshold_otsu(img), min_thr)
        except:
            threshold = img.min() + min_thr
    img[img < threshold] = threshold
    img = img - threshold
    return(img)
def sub_bg_thr_z(imgz, min_thr = 0, threshold = None):
    for i in range(len(imgz)):
        imgz[i] = sub_bg_thr(imgz[i], min_thr, threshold)
    return(imgz)
def threshold_z(imgz):
    thresholds = [filters.threshold_otsu(img) for img in imgz]
    return(max(thresholds))
def threshold_all(imgs):
    thresholds = [max([filters.threshold_otsu(img) for img in imgz]) for imgz in imgs]
    return(max(thresholds)) 
def remove_background(img, rolling_ball_radius = 25):
    background = restoration.rolling_ball(img, radius=rolling_ball_radius)
    rolling_ball_filtered = img - background
    return(rolling_ball_filtered)
def remove_background_z(imgz, rolling_ball_radius = 25):
    for i in range(len(imgz)):
        imgz[i] = remove_background(imgz[i], rolling_ball_radius)
    return(imgz)
def denoise(img, min_thr = 0, rolling_ball_filtered = 25, threshold = None):
    return(filter(remove_background(img, rolling_ball_filtered), min_thr, threshold))
def denoise_z(imgz, min_thr = 0, rolling_ball_radius = 25, threshold = None):
    imgz_denoised = copy.deepcopy(imgz)
    try:
        for i in range(len(imgz)):
            imgz_denoised[i] = remove_background(imgz[i], rolling_ball_radius)
        if threshold == None:
            threshold = threshold_z(imgz_denoised)
        for i in range(len(imgz)):
            imgz_denoised[i] = filter(imgz_denoised[i], min_thr, threshold)
    except:
        # imgz_denoised = normalize(imgz)
        print("Denoising failed, returning the original image")
    return(imgz_denoised)

In [29]:
def max_z(imgz):
    try:
        img_sum = np.max(imgz, axis=0)
    except:
        img_sum = np.zeros_like(imgz)
    return(img_sum)
def sum_z(imgz):
    try:
        img_sum = np.sum(imgz, axis=0)
    except:
        img_sum = np.zeros_like(imgz)
    return(img_sum)
def ave_z(imgz):
    try:
        img_ave = np.mean(imgz, axis=0)
    except:
        img_ave = np.zero_like(imgz)
    return(img_ave)

In [30]:
def simple_stitch(imgs, seq_num = 4, hr_margin = 233, vu_margin = 0):
    images = []
    for i in range(seq_num):
        img = imgs[i]
        if i < 3:
            images.append(img[:,:-hr_margin])
        else:
            images.append(img)
    img1 = np.hstack(images)
    images = []
    for i in range(seq_num):
        img = imgs[i+seq_num]
        if i < 3:
            images.append(img[:,:-hr_margin])
        else:
            images.append(img)
    img2 = np.hstack(images)
    im_h = np.vstack([img1[vu_margin:, :], img2[vu_margin:, :]])
    return(im_h)
def simple_stitch_z(imgs, z_num = None, seq_num = 4, hr_margin = 233, vu_margin = 0):
    if z_num == None:
        z_num = len(imgs[0])
    stitched = []
    for i in range(z_num):
        images = [img[i] for img in imgs]
        stitched.append(simple_stitch(images, seq_num, hr_margin, vu_margin))
    return(np.array(stitched))

def simple_stitch_alt(imgs, seq_num = 4, hr_margin = 233, vu_margin = 0):
    images = []
    for i in range(seq_num):
        img = imgs[i * 2]
        if i < 3:
            images.append(img[:,:-hr_margin])
        else:
            images.append(img)
    img1 = np.hstack(images)
    images = []
    for i in range(seq_num):
        img = imgs[i * 2 + 1]
        if i < 3:
            images.append(img[:,:-hr_margin])
        else:
            images.append(img)
    img2 = np.hstack(images)
    im_h = np.vstack([img1[vu_margin:, :], img2[vu_margin:, :]])
    return(im_h)
def simple_stitch_z_alt(imgs, z_num = None, seq_num = 4, hr_margin = 233, vu_margin = 0):
    if z_num == None:
        z_num = len(imgs[0])
    stitched = []
    for i in range(z_num):
        images = [img[i] for img in imgs]
        stitched.append(simple_stitch_alt(images, seq_num, hr_margin, vu_margin))
    return(np.array(stitched))

In [31]:
def sharping(img, cut_off = None):
    if cut_off == None:
        cut_off = int(img.max() / 2)
    else:
        cut_off = int(img.max() * cut_off)
    img[img <= cut_off] = 0
    img[img > cut_off] = img.max()
    return(img)
def sum_y(img):
    img1d = np.sum(img, axis=0)
    return(img1d)
def slide_sum_y(img, slide_len = 100):
    img1d = np.sum(img, axis=0)
    img_slides = []
    for i in range(0, len(img1d) - slide_len):
        img_slides.append(np.sum(img1d[i:i+slide_len], axis=0))
    return(np.array(img_slides))
def get_pop_density(img1d):
    pop_density = img1d/img1d.sum(axis=1, keepdims=True)
    return(pop_density)

In [32]:
def get_mask(img):
    img[img > 0] = 1
    img = np.abs(1 - img)
    return(img)
def get_mask_inv(img):
    img[img > 0] = 1
    return(img)
def mask_img(img, mask):
    img[mask == 0] = 0
    return(img)
def simple_stitch_z_with_mask(imgs, mask, z_num = None, seq_num = 4, hr_margin = 233, vu_margin = 0):
    if z_num == None:
        z_num = len(imgs[0])
    stitched = []
    for i in range(z_num):
        images = [img[i] for img in imgs]
        stitched.append(mask_img(simple_stitch(images, seq_num, hr_margin, vu_margin), get_mask(mask)))
    return(np.array(stitched))

## Set directories

In [33]:
work_dir = "/Users/feng626/workspace/data/SoilSFA/spatial_interactions"

In [34]:
rep3_dir = work_dir + "/Activity_1.3_rep3_02.07.22_start"
rep3_img_dirs = [name for name in os.listdir(rep3_dir) if os.path.isdir(os.path.join(rep3_dir, name))] 
rep3_img_dirs.sort()

In [35]:
rep2_dir = work_dir + "/Activity_1.3_rep2_12.13.21_start"
rep2_img_dirs = [name for name in os.listdir(rep2_dir) if os.path.isdir(os.path.join(rep2_dir, name))] 
rep2_img_dirs.sort()

In [36]:
patch_number = 8

## Work on the rep3

In [37]:
# # Stitch all the BF figures and keep z stacks
# for chi, channel in enumerate(['w1SD BF', 'w2SD DAPI', 'w3SD GFP', 'w4SD RFP']):
#     for dir in rep3_img_dirs:
#         if dir == "02.09.22_11AM_Chito5" and chi != 0:
#             continue
#         img_dir = rep3_dir + '/' + dir 
#         tif_files = [img_dir + '/' + dir + '_1_' + channel +  '_s' + str(i+1) + '.TIF' for i in range(patch_number)]
#         tif_files.sort()
#         tif_file_new = img_dir + '/' + dir + '_1_{}_stitched.TIF'.format(channel)
#         # if not os.path.isfile(tif_file_new):
#         imgs = [io.imread(tif_file) for tif_file in tif_files]
#         if chi == 0:
#             imgzn = simple_stitch(imgs)
#         else:
#             imgzn = simple_stitch_z(imgs)
#         io.imsave(tif_file_new, imgzn)
#         # else:
#         #     imgs = [io.imread(tif_file) for tif_file in tif_files]
#         #     if chi == 0:
#         #         imgzn = simple_stitch_alt(imgs)
#         #     else:
#         #         imgzn = simple_stitch_z_alt(imgs)
#         #     io.imsave(tif_file_new[:-4] + "_alt.TIF", imgzn)

In [38]:
# for chi, channel in enumerate(['w2SD DAPI', 'w3SD GFP', 'w4SD RFP']):
#     for dir in rep3_img_dirs:
#         if dir == "02.09.22_11AM_Chito5":
#             continue
#         img_dir = rep3_dir + '/' + dir 
#         for i in range(patch_number):
#             tif_file = img_dir + '/' + dir + '_1_{}_s{}.TIF'.format(channel, i+1)
#             tif_file_new = img_dir + '/' + dir + '_1_{}_s{}_wo_bg.TIF'.format(channel, i+1)
#             if not os.path.isfile(tif_file_new):
#                 imgz = io.imread(tif_file)
#                 imgzn = remove_background_z(imgz)
#                 io.imsave(tif_file_new, imgzn)

In [39]:
min_thr = 32
result_dir = work_dir + "/results/results_rep3"
for dir in rep3_img_dirs:
    if dir == "02.09.22_11AM_Chito5":
        continue
    img_dir = rep3_dir + '/' + dir 
    all_channels = []
    all_channels_ori = []
    for chi, channel in enumerate(['w4SD RFP', 'w3SD GFP', 'w2SD DAPI']):
        masked_file = img_dir + '/' + dir + '_1_w1SD BF_stitched_mask.tif'
        tif_files = [img_dir + '/' + dir + '_1_' + channel +  '_s' + str(i+1) + '.TIF' for i in range(patch_number)]
        tif_files.sort()
        imgn_filtered_tif_file= result_dir + '/' + dir + '_1_{}_processed.TIF'.format(channel)
        imgn_sharp_tif_file= result_dir + '/' + dir + '_1_{}_processed_sharp.TIF'.format(channel)
        projected_csv_file = result_dir + '/' + dir + '_1_{}_processed.csv'.format(channel)

        imgs_mask = get_mask(io.imread(masked_file))
        imgs = [io.imread(tif_file) for tif_file in tif_files]
        imgz = simple_stitch_z(imgs)
        imgz_rm_bg = remove_background_z(imgz)
        imgz_filtered = filter_z(imgz_rm_bg)  # Don't use the minimal threshold
        masked_imgz = np.array([mask_img(imgzi, imgs_mask) for imgzi in imgz_filtered])
        imgp = max_z(masked_imgz)
        imgp_rm_bg = remove_background_z(imgp)
        imgp_filtered = filter(imgp_rm_bg, min_thr)

        img_to_merge = normalize(imgp_filtered)
        img_to_merge = remove_background_z(img_to_merge)
        img_to_merge = filter(img_to_merge, min_thr)
        tifffile.imsave(imgn_filtered_tif_file, img_to_merge)
        np.savetxt(projected_csv_file, img_to_merge, delimiter=',')
        img2m_sharp = sharping(img_to_merge, 0.01)
        tifffile.imsave(imgn_sharp_tif_file, img2m_sharp)

        all_channels_ori.append(img_to_merge)
        all_channels.append(img2m_sharp)
        
    combined_img_ori = np.transpose(all_channels_ori, (1,2,0))
    combined_img = np.transpose(all_channels, (1,2,0))
        
    output = result_dir + '/' + dir + '_1_combined_processed'
    img_out_ori = output + '.TIF'
    img_out = output + '_sharp.TIF'
    img_out_small_ori = output + '_small.TIF'
    img_out_small = output + '_small_sharp.TIF'
    pkl_out = output + '.pkl'
    plot_out_ori = output + '_1d.png'
    plot_out = output + '_1d_sharp.png'
    
    tifffile.imsave(img_out_ori, combined_img_ori)
    tifffile.imsave(img_out, combined_img)

    with open(pkl_out, 'wb') as f:
        pickle.dump(combined_img_ori, f)
        
    plt.figure(figsize=(16,8))
    plt.imshow(combined_img_ori)
    plt.savefig(img_out_small_ori, dpi=200)
    plt.close()

    plt.figure(figsize=(16,8))
    plt.imshow(combined_img)
    plt.savefig(img_out_small, dpi=200)
    plt.close()

    pop_den = get_pop_density(slide_sum_y(combined_img_ori, combined_img_ori.shape[1]//20))
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(16,8))
    for i in range(3):
        plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
    # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
    plt.savefig(plot_out_ori, dpi=200)
    # plt.show()
    plt.close()

    pop_den = get_pop_density(slide_sum_y(combined_img, combined_img.shape[1]//20))
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(16,8))
    for i in range(3):
        plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
    # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
    plt.savefig(plot_out, dpi=200)
    # plt.show()
    plt.close()

## Work on the rep2

In [ ]:
# # Stitch all the BF figures and keep z stacks
# for chi, channel in enumerate(['w1SD BF', 'w2SD DAPI', 'w3SD GFP', 'w4SD RFP']):
#     for dir in rep2_img_dirs:
#         img_dir = rep2_dir + '/' + dir 
#         tif_files = [img_dir + '/' + dir + '_1_' + channel +  '_s' + str(i+1) + '.TIF' for i in range(patch_number)]
#         tif_files.sort()
#         tif_file_new = img_dir + '/' + dir + '_1_{}_stitched.TIF'.format(channel)
#         # if not os.path.isfile(tif_file_new):
#         imgs = [io.imread(tif_file) for tif_file in tif_files]
#         if chi == 0:
#             imgzn = simple_stitch(imgs)
#         else:
#             imgzn = simple_stitch_z(imgs)
#         io.imsave(tif_file_new, imgzn)
#         # else:
#         #     imgs = [io.imread(tif_file) for tif_file in tif_files]
#         #     if chi == 0:
#         #         imgzn = simple_stitch_alt(imgs)
#         #     else:
#         #         imgzn = simple_stitch_z_alt(imgs)
#         #     io.imsave(tif_file_new[:-4] + "_alt.TIF", imgzn)

In [ ]:
# for chi, channel in enumerate(['w2SD DAPI', 'w3SD GFP', 'w4SD RFP']):
#     for dir in rep2_img_dirs:
#         img_dir = rep2_dir + '/' + dir 
#         for i in range(patch_number):
#             tif_file = img_dir + '/' + dir + '_1_{}_s{}.TIF'.format(channel, i+1)
#             tif_file_new = img_dir + '/' + dir + '_1_{}_s{}_wo_bg.TIF'.format(channel, i+1)
#             if not os.path.isfile(tif_file_new):
#                 imgz = io.imread(tif_file)
#                 imgzn = remove_background_z(imgz)
#                 io.imsave(tif_file_new, imgzn)

In [ ]:
min_thr = 32
result_dir = work_dir + "/results/results_rep2"
for dir in rep2_img_dirs:
    img_dir = rep2_dir + '/' + dir 
    all_channels = []
    all_channels_ori = []
    for chi, channel in enumerate(['w4SD RFP', 'w3SD GFP', 'w2SD DAPI']):
        masked_file = img_dir + '/' + dir + '_1_w1SD BF_stitched_mask.tif'
        tif_files = [img_dir + '/' + dir + '_1_' + channel +  '_s' + str(i+1) + '.TIF' for i in range(patch_number)]
        tif_files.sort()
        imgn_filtered_tif_file= result_dir + '/' + dir + '_1_{}_processed.TIF'.format(channel)
        imgn_sharp_tif_file= result_dir + '/' + dir + '_1_{}_processed_sharp.TIF'.format(channel)
        projected_csv_file = result_dir + '/' + dir + '_1_{}_processed.csv'.format(channel)

        imgs_mask = get_mask(io.imread(masked_file))
        imgs = [io.imread(tif_file) for tif_file in tif_files]
        imgz = simple_stitch_z(imgs)
        imgz_rm_bg = remove_background_z(imgz)
        imgz_filtered = filter_z(imgz_rm_bg)  # Don't use the minimal threshold
        masked_imgz = np.array([mask_img(imgzi, imgs_mask) for imgzi in imgz_filtered])
        imgp = max_z(masked_imgz)
        imgp_rm_bg = remove_background_z(imgp)
        imgp_filtered = filter(imgp_rm_bg, min_thr)

        img_to_merge = normalize(imgp_filtered)
        img_to_merge = remove_background_z(img_to_merge)
        img_to_merge = filter(img_to_merge, min_thr)
        tifffile.imsave(imgn_filtered_tif_file, img_to_merge)
        np.savetxt(projected_csv_file, img_to_merge, delimiter=',')
        img2m_sharp = sharping(img_to_merge, 0.01)
        tifffile.imsave(imgn_sharp_tif_file, img2m_sharp)

        all_channels_ori.append(img_to_merge)
        all_channels.append(img2m_sharp)
        
    combined_img_ori = np.transpose(all_channels_ori, (1,2,0))
    combined_img = np.transpose(all_channels, (1,2,0))
        
    output = result_dir + '/' + dir + '_1_combined_processed'
    img_out_ori = output + '.TIF'
    img_out = output + '_sharp.TIF'
    img_out_small_ori = output + '_small.TIF'
    img_out_small = output + '_small_sharp.TIF'
    pkl_out = output + '.pkl'
    plot_out_ori = output + '_1d.png'
    plot_out = output + '_1d_sharp.png'
    
    tifffile.imsave(img_out_ori, combined_img_ori)
    tifffile.imsave(img_out, combined_img)

    with open(pkl_out, 'wb') as f:
        pickle.dump(combined_img_ori, f)
        
    plt.figure(figsize=(16,8))
    plt.imshow(combined_img_ori)
    plt.savefig(img_out_small_ori, dpi=200)
    plt.close()

    plt.figure(figsize=(16,8))
    plt.imshow(combined_img)
    plt.savefig(img_out_small, dpi=200)
    plt.close()

    pop_den = get_pop_density(slide_sum_y(combined_img_ori, combined_img_ori.shape[1]//20))
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(16,8))
    for i in range(3):
        plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
    # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
    plt.savefig(plot_out_ori, dpi=200)
    # plt.show()
    plt.close()

    pop_den = get_pop_density(slide_sum_y(combined_img, combined_img.shape[1]//20))
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(16,8))
    for i in range(3):
        plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
    # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
    plt.savefig(plot_out, dpi=200)
    # plt.show()
    plt.close()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Working on rep1

In [ ]:
min_thr = 32
image_dir = "/Users/feng626/workspace/data/SoilSFA/spatial_interactions/Activity_1.3_rep1_11.18.21_start"
result_dir = work_dir + "/results/results_rep1"
for carbon in ["Chitin", "Chito5", "Nag"]:
# for carbon, timepoints in zip(["Chitin", "Chito5", "Nag"], [["11.22.21_17"], ["11.22.21_16"], ["11.22.21_15"]]):
    time_dir = image_dir + '/' + carbon 
    timepoints = [name for name in os.listdir(time_dir) if os.path.isdir(os.path.join(time_dir, name))] 
    timepoints.sort()
    for timepoint in timepoints:
        all_channels = []
        for chi, channel in enumerate(['w4SD RFP', 'w3SD GFP', 'w2SD DAPI']):
            masked_file = image_dir + '/' + carbon + '/' + timepoint + '/' + timepoint + '_' + carbon + '_1_w1SD BF_stitched_MASK.tif'
            tif_files = [image_dir + '/' + carbon + '/' + timepoint + '/' + timepoint + '_' + carbon + '_1_' + channel + '_s' + str(i+1) + '.TIF' for i in range(patch_number)]
            tif_files.sort()
            imgn_filtered_tif_file= result_dir + '/' + timepoint + '_' + carbon + '_1_{}_processed.TIF'.format(channel)
            imgn_sharp_tif_file= result_dir + '/' + timepoint + '_' + carbon + '_1_{}_processed_sharp.TIF'.format(channel)
            projected_csv_file = result_dir + '/' + timepoint + '_' + carbon + '_1_{}_processed.csv'.format(channel)

            imgs_mask = get_mask(io.imread(masked_file))
            imgs = [io.imread(tif_file) for tif_file in tif_files]
            imgz = simple_stitch_z(imgs)
            imgz_rm_bg = remove_background_z(imgz)
            imgz_filtered = filter_z(imgz_rm_bg)  # Don't use the minimal threshold
            masked_imgz = np.array([mask_img(imgzi, imgs_mask) for imgzi in imgz_filtered])
            imgp = max_z(masked_imgz)
            imgp_rm_bg = remove_background_z(imgp)
            imgp_filtered = filter(imgp_rm_bg, min_thr)
    
            img_to_merge = normalize(imgp_filtered)
            img_to_merge = remove_background_z(img_to_merge)
            img_to_merge = filter(img_to_merge, min_thr)
            tifffile.imsave(imgn_filtered_tif_file, img_to_merge)
            np.savetxt(projected_csv_file, img_to_merge, delimiter=',')
            img2m_sharp = sharping(img_to_merge, 0.01)
            tifffile.imsave(imgn_sharp_tif_file, img2m_sharp)
    
            all_channels_ori.append(img_to_merge)
            all_channels.append(img2m_sharp)
            
        combined_img_ori = np.transpose(all_channels_ori, (1,2,0))
        combined_img = np.transpose(all_channels, (1,2,0))
            
        output = result_dir + '/' + timepoint + '_' + carbon + '_1_combined_processed'
        img_out_ori = output + '.TIF'
        img_out = output + '_sharp.TIF'
        img_out_small_ori = output + '_small.TIF'
        img_out_small = output + '_small_sharp.TIF'
        pkl_out = output + '.pkl'
        plot_out_ori = output + '_1d.png'
        plot_out = output + '_1d_sharp.png'
        
        tifffile.imsave(img_out_ori, combined_img_ori)
        tifffile.imsave(img_out, combined_img)
    
        with open(pkl_out, 'wb') as f:
            pickle.dump(combined_img_ori, f)
            
        plt.figure(figsize=(16,8))
        plt.imshow(combined_img_ori)
        plt.savefig(img_out_small_ori, dpi=200)
        plt.close()
    
        plt.figure(figsize=(16,8))
        plt.imshow(combined_img)
        plt.savefig(img_out_small, dpi=200)
        plt.close()
    
        pop_den = get_pop_density(slide_sum_y(combined_img_ori, combined_img_ori.shape[1]//20))
        colors = ['red', 'green', 'blue']
        plt.figure(figsize=(16,8))
        for i in range(3):
            plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
        # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
        plt.savefig(plot_out_ori, dpi=200)
        # plt.show()
        plt.close()
    
        pop_den = get_pop_density(slide_sum_y(combined_img, combined_img.shape[1]//20))
        colors = ['red', 'green', 'blue']
        plt.figure(figsize=(16,8))
        for i in range(3):
            plt.plot(range(len(pop_den)), pop_den[:,i], color = colors[i], linewidth=3)
        # plt.plot(range(len(pop_den)), pop_den, linewidth=2)
        plt.savefig(plot_out, dpi=200)
        # plt.show()
        plt.close()